## kerashw08
同样的数据，用lstm来完成情感分类

In [1]:
import pandas as pd 
import numpy as np 
import jieba # pip install jieba

from keras.layers import Dense, Input, Flatten, Dropout
from keras.layers import Conv1D, MaxPooling1D, Embedding, concatenate
from keras.layers.recurrent import LSTM
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model, Sequential

C:\Users\Ian\Anaconda3\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
# 读入数据
neg=pd.read_excel('data/neg.xls',header=None)
pos=pd.read_excel('data/pos.xls',header=None)

In [3]:
#合并语料
pn = pd.concat([pos,neg],ignore_index=True) 
#计算语料数目
neglen = len(neg)
print(neglen)
poslen = len(pos) 
print(poslen)

10428
10677


In [6]:
pn.head(1)

,0
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...


In [7]:
#定义分词函数
cw = lambda x: list(jieba.cut(x))
pn['words'] = pn[0].apply(cw)
pn.head(1)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\Ian\AppData\Local\Temp\jieba.cache
Loading model cost 0.881 seconds.
Prefix dict has been built succesfully.


,0,words
0,做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一...,"[做, 父母, 一定, 要, 有, 刘墉, 这样, 的, 心态, ，, 不断, 地, 学习,..."


In [5]:
# 一行数据最多的词汇数
max_document_length = max([len(x) for x in pn['words']])
max_document_length

1804

In [8]:
# 设置一个评论最多1000个词
max_document_length = 1000

In [9]:
texts = [' '.join(x) for x in pn['words']]

In [10]:
# 实例化分词器，设置字典中最大词汇数为30000
tokenizer = Tokenizer(num_words=30000)
# 传入我们的训练数据，建立词典
tokenizer.fit_on_texts(texts) 
# 把词转换为编号，词的编号根据词频设定，频率越大，编号越小
sequences = tokenizer.texts_to_sequences(texts) 

In [12]:
sequences[:1]

[[130,
  503,
  241,
  40,
  12,
  12468,
  83,
  2,
  1599,
  1,
  922,
  210,
  335,
  1,
  922,
  210,
  2309,
  1,
  922,
  210,
  39,
  36,
  529,
  1,
  37,
  36,
  1566,
  3192,
  1679,
  2,
  1017,
  3,
  6,
  84,
  1,
  15,
  5,
  49,
  60,
  8,
  20,
  2,
  17,
  63,
  1415,
  2,
  29,
  530,
  3436,
  3,
  168,
  12468,
  2,
  648,
  1,
  5204,
  37,
  6,
  119,
  29,
  669,
  2,
  10949,
  2,
  3193,
  1,
  49,
  1600,
  1136,
  10,
  17,
  63,
  699,
  2,
  4815,
  1,
  39,
  63,
  2719,
  98,
  29,
  937,
  199,
  17,
  1118,
  2,
  164,
  244,
  3,
  8,
  43,
  12468,
  10,
  4313,
  8991,
  2,
  130,
  503,
  2,
  521,
  87,
  1,
  37,
  23,
  414,
  8260,
  1,
  503,
  17,
  4816,
  766,
  1083,
  9,
  5,
  307,
  8992,
  1,
  21850,
  21851,
  1095,
  5003,
  4,
  1,
  8993,
  21852,
  80,
  113,
  12,
  4314,
  3,
  112,
  1,
  130,
  503,
  2,
  116,
  21853,
  4,
  1,
  546,
  816,
  546,
  1472,
  4049,
  53,
  21854,
  1,
  17330,
  1,
  335,
  3347,
  3,
  3826,

In [14]:
# 把序列设定为1000的长度，超过1000的部分舍弃，不到1000则补0
sequences = pad_sequences(sequences, maxlen=1000, padding='post')  
sequences = np.array(sequences)

In [15]:
sequences.shape

(21105, 1000)

In [16]:
# 词对应编号的字典
dict_text = tokenizer.word_index
dict_text['也']

9

In [20]:
# 定义标签
positive_labels = [1 for _ in range(poslen)]
negative_labels = [0 for _ in range(neglen)]
y = np.concatenate([positive_labels, negative_labels], 0)

# 打乱数据
np.random.seed(10)
shuffle_indices = np.random.permutation(np.arange(len(y)))
x_shuffled = sequences[shuffle_indices]
y_shuffled = y[shuffle_indices]

# 数据集切分为两部分
test_sample_index = -1 * int(0.1 * float(len(y)))
x_train, x_test = x_shuffled[:test_sample_index], x_shuffled[test_sample_index:]
y_train, y_test = y_shuffled[:test_sample_index], y_shuffled[test_sample_index:]

In [29]:
# 定义函数式模型
# 模型输入
sequence_input = Input(shape=(1000,))
# Embedding层，30000表示30000个词，每个词对应的向量为128维，序列长度为1000
embedding_layer = Embedding(30000,
                            128,
                            input_length=1000)
embedded_sequences = embedding_layer(sequence_input)

lstm = LSTM(64, return_sequences=True)(embedded_sequences)

x = Flatten()(lstm)
# 输出层
preds = Dense(1, activation='sigmoid')(x)
# 定义模型
model = Model(sequence_input, preds)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_8 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_10 (Embedding)     (None, 1000, 128)         3840000   
_________________________________________________________________
lstm_10 (LSTM)               (None, 1000, 64)          49408     
_________________________________________________________________
flatten_1 (Flatten)          (None, 64000)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 64001     
Total params: 3,953,409
Trainable params: 3,953,409
Non-trainable params: 0
_________________________________________________________________


In [30]:
# 训练模型
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Train on 18995 samples, validate on 2110 samples
Epoch 1/5
18995/18995 [==============================] - 228s 12ms/step - loss: 0.3670 - acc: 0.8235 - val_loss: 0.2258 - val_acc: 0.9104
Epoch 2/5
18995/18995 [==============================] - 227s 12ms/step - loss: 0.1314 - acc: 0.9556 - val_loss: 0.1958 - val_acc: 0.9284
Epoch 3/5
18995/18995 [==============================] - 224s 12ms/step - loss: 0.0699 - acc: 0.9770 - val_loss: 0.1984 - val_acc: 0.9318
Epoch 4/5
18995/18995 [==============================] - 226s 12ms/step - loss: 0.0436 - acc: 0.9878 - val_loss: 0.2796 - val_acc: 0.9284
Epoch 5/5
18995/18995 [==============================] - 226s 12ms/step - loss: 0.0311 - acc: 0.9917 - val_loss: 0.2460 - val_acc: 0.9275


## 采用bilstm
效果应该是最好的，不过最后发生过拟合了！
![](bilstm.png)

In [34]:
from keras.layers.wrappers import Bidirectional
# 定义函数式模型
# 模型输入
sequence_input = Input(shape=(1000,))
# Embedding层，30000表示30000个词，每个词对应的向量为128维，序列长度为1000
embedding_layer = Embedding(30000,
                            128,
                            input_length=1000)
embedded_sequences = embedding_layer(sequence_input)

lstm = Bidirectional(LSTM(64, return_sequences=True))(embedded_sequences)

x = Flatten()(lstm)
# 输出层
preds = Dense(1, activation='sigmoid')(x)
# 定义模型
model = Model(sequence_input, preds)

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_9 (InputLayer)         (None, 1000)              0         
_________________________________________________________________
embedding_11 (Embedding)     (None, 1000, 128)         3840000   
_________________________________________________________________
bidirectional_1 (Bidirection (None, 1000, 128)         98816     
_________________________________________________________________
flatten_2 (Flatten)          (None, 128000)            0         
_________________________________________________________________
dense_13 (Dense)             (None, 1)                 128001    
Total params: 4,066,817
Trainable params: 4,066,817
Non-trainable params: 0
_________________________________________________________________


In [35]:
# 训练模型
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['acc'])

model.fit(x_train, y_train,
          batch_size=128,
          epochs=5,
          validation_data=(x_test, y_test))

Train on 18995 samples, validate on 2110 samples
Epoch 1/5
18995/18995 [==============================] - 514s 27ms/step - loss: 0.3768 - acc: 0.8204 - val_loss: 0.1980 - val_acc: 0.9299
Epoch 2/5
18995/18995 [==============================] - 511s 27ms/step - loss: 0.1259 - acc: 0.9551 - val_loss: 0.1808 - val_acc: 0.9370
Epoch 3/5
18995/18995 [==============================] - 513s 27ms/step - loss: 0.0676 - acc: 0.9777 - val_loss: 0.2160 - val_acc: 0.9336
Epoch 4/5
18995/18995 [==============================] - 513s 27ms/step - loss: 0.0394 - acc: 0.9880 - val_loss: 0.2790 - val_acc: 0.9284
Epoch 5/5
18995/18995 [==============================] - 519s 27ms/step - loss: 0.0263 - acc: 0.9939 - val_loss: 0.3060 - val_acc: 0.9251
